In [1]:
#Test Conditions
avg_dem = 3
adi = 1.24
cv2 = 0.26
lev = avg_dem * adi
T = 60      #Number of time periods
G = 3       #Number of products
ncust = 2
nsup = 3
obs = 180
nreal = 50 # number of realizations to generate
LT = 7 #based on LT of resupply from TM to supplier
N=1000 # Number of replications to generate LTD based on
AM_capacity = 4 
ndesign = 16


16

In [2]:
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/1. Tradeoff Curvers and Value of AM Addition/2. Scripts/Generate_Forecast.jl")

AM_design

In [34]:
######### All generate forecast code
hist_dem = gen_historical(adi, cv2, lev, G, ncust)
dem_real = gen_dem_realization(nreal, adi, cv2, lev, T, G, ncust);
forecast = forecast_dem(hist_dem,G,T)
(LTD, Q, pAll) = little_s_inventory(nsup, ncust, LT, N, hist_dem)
NetDesign = AM_design(nsup,ncust)
ndesign = size(NetDesign)[1]

idx_Croston = [1, 2]
idx_SBA = Int64[]
idx_Croston = [1, 2]
idx_SBA = Int64[]
idx_Croston = [1]
idx_SBA = [2]


LoadError: BoundsError: attempt to access 1-element Array{Float64,1} at index [2]

In [38]:
sum(dem_real,dims=2)

50×1×2×3 Array{Int64,4}:
[:, :, 1, 1] =
 157
 163
 213
 185
 152
 170
 164
 147
 186
 196
 173
 198
 195
   ⋮
 163
 171
 207
 178
 195
 192
 205
 175
 180
 160
 178
 193

[:, :, 2, 1] =
 192
 168
 195
 176
 193
 174
 168
 179
 201
 172
 204
 181
 209
   ⋮
 159
 181
 189
 189
 200
 155
 190
 169
 161
 182
 183
 188

[:, :, 1, 2] =
 174
 183
 190
 174
 186
 200
 204
 186
 174
 198
 172
 195
 174
   ⋮
 176
 178
 180
 177
 189
 181
 168
 195
 190
 186
 186
 217

[:, :, 2, 2] =
 178
 187
 188
 193
 188
 200
 182
 191
 182
 198
 222
 155
 210
   ⋮
 174
 206
 173
 155
 204
 190
 164
 202
 175
 195
 186
 209

[:, :, 1, 3] =
 191
 177
 165
 201
 171
 185
 202
 180
 185
 181
 189
 209
 187
   ⋮
 174
 149
 189
 159
 206
 170
 190
 181
 199
 189
 171
 171

[:, :, 2, 3] =
 180
 188
 210
 189
 199
 162
 186
 175
 188
 213
 180
 167
 205
   ⋮
 158
 170
 170
 167
 170
 201
 151
 169
 172
 178
 180
 184

In [19]:
#Packages needed
using Optim
using Plots
using StatsBase
using Distributions
using Statistics
using Random
using JuMP, Gurobi
using PrettyTables
using Distributed
using TickTock

In [9]:

"""
Coefficient Description
Used in Obj Function
    CpTM = Cost of Production Tradition Manufacturing (TM) - NumTM x T x G matrix
    CpAM = Cost of Production Additive Manufacturing (AM) - NumPossAMlocs x T x G matrix
    CtTM = Cost of Transportation from TM to Supplier (DCs) - NumTM2DCroutes x T x G matrix 
    CtS = Cost of Transportation from Supplier to SLs - NumDCs x (ncust*T) x G matrix
    Ci = Inventory Cost (at supplier) - nsup x T+1 x G matrix
    CiRetailer = Inventory Cost (at SLs) - ncsut x T+1 x G matrix
    CBO = Backorder Costs - ncust x T+1 x G matrix
Other Coefs
    Creorder
    cdel
    CpExp, CtExp = Expedited production + transportation costs used in Eval (TM to SL)
"""


### Need lags of TM, 

LagTM = [7 7 7] # Rows = TM Columns = Sup 
LagS = [1 1;2 2;1 1] # Rows = Sup  Columns = Cust (3 Sup 2 Cust)

# Production Cost TM
nTM = 1 # 1x TM per product. This should never change based on my assumption that only 1x TM supplies a product
CpTM = []
for g in 1:G
    push!(CpTM, 1) # Rows = TM . Colums = Time periods. 3D: products
end

CpTM = repeat(CpTM,1,T)

# Assuming CtTM is same for both TM. If different change this. ASSUMPTION:
# products come from the same location.
CtTM = [25 25 25] # . Transport from TM to supplier location. Rows = Sup locations. 3D = Product (new TM)

#CtTM = repeat(CtTM',1,T,G) # Columns turn to time. 

# Valid arc connections for TM - Sup. (Assumption: TM services all
# locations for all products. These should always be 1. This is more of a
# future capability
valid_TMrte = zeros(G,1,3)
valid_TMrte[:,:,1] = [1 1 1]' # Product 1 valid rte
valid_TMrte[:,:,2] = [1 1 1]' # product 2 valid rte
valid_TMrte[:,:,3] = [1 1 1]' # Product 3 valid rte
# valid_TMrte_prod4 = [1 1 1] # product 4 valid rte

valid_TMrte = repeat(valid_TMrte.*Inf,1,T,1)
valid_TMrte[isnan.(valid_TMrte)] .= 0   

CtS = [25 25; 50 50; 25 25] # Rows = Sup. Columns = Cust. 3D = Product

CtSexpanded = repeat(CtS,1,T,G)

# Set which arc connections are actually valid (aka which suppliers connect
# to which customers) (force the uB to be zero for not valid rte
# connetions) This is used in both MILP
Suprte = [1 0; 1 1; 0 1] # Valid arc is 1. NOT valid = 0
valid_Suprte = zeros(nsup,T,G,ncust)
for i=1:nsup
    for j=1:ncust
        valid_Suprte[i,:,:,j] .= (Suprte[i,j] * Inf)
    end
end
valid_Suprte[isnan.(valid_Suprte)].=0 # replace NaNs with 0
valid_Suprte

# All below Needs to be a function of value of the product. Wont be the same for all products
Ci = [1; 1; 1] # Rows = Sup. Columns = Time. 3D = Prod
Ci = repeat(Ci,1,T+1,G)
Ci[:,1,:] .= 0  # Intital inventory cost already accounted for last period 

CiRetailer = [4; 5] # Rows = Retailer. Columns = Time. 3D = period
CiRetailer = repeat(CiRetailer,1,T+1,G)
CiRetailer[:,1,:] .= 0

CBO = [378; 378]    # Rows = Sup Colums = T 3D = Prod
CBO = repeat(CBO,1,T+1,G)

#*************** CpAM incorporates both supplier and customers because AM is located at both locations
CpAM = [1; 1; 1; 1 ;1] # Rows = Sup and Cust. Suppliers must be first. Columns = Time. 3D = Products
CpAM = repeat(CpAM,1,T,G)
##
## Auxillary Variables

Creorder = 0
Creorder = repeat([Creorder],nsup,T,G) # Rows

cdel = 0
cdel = repeat([cdel],ncust,T,G) # Rows = Customer. Colums = Time. 3D = rProduct

## This is where you can either fix your policies or set the lower and upper bounds on them.
##*** CURRENTLY WE USE CALCULATED LB OF MIN POLICY AND MAX POLICY*** Insert this if you want
##to hard code them in.
# lbMinPolicy1 = [1 1 1]' # Prodcut 1: For each supplier. Rows = Sup. Column = prod
# lbMinPolicy2 = [1 1 1]' # Prodcut 2: For each supplier
# lbMinPolicy3 = [1 1 1]' # Prodcut 3: For each supplier
# # lbMinPolicy4 = [4 4 4]' # Prodcut 4: For each supplier
# lbMinPolicy = cat(3,lbMinPolicy1, lbMinPolicy2,lbMinPolicy3)
# 
# lbMaxPolicy1 = [2 2 2]'
# lbMaxPolicy2 = [2 2 2]'
# lbMaxPolicy3 = [2 2 2]'
# # lbMaxPolicy4 = [8 8 8]'
# 
# lbMaxPolicy = cat(3,lbMaxPolicy1,lbMaxPolicy2,lbMaxPolicy3)


##
M = 1000 # Big M
## Expedited Production and Expedited Transport is not used in the true MILP but it is used in Eval MILP.
# Cost of Expedited Production (In reality it takes the same amount of time
# to produce, so the cost should just be the same as CpTM?)
CpExp = Array{Int64}(undef,1,1,3)
CpExp[1,:,1] .= 1 # Product 1
CpExp[1,:,2] .= 1 # Prodcut 2
CpExp[1,:,3] .= 1 # Product 3
CpExp = repeat(CpExp,1,T,1)

# Cost of expedited transportation. Transport from TM direct to RETAILER.
# (ONLY USED IN EVAL CODE)
CtExp = [100 100];# Transport from TM to RETAILER DIRECT location. Rows = RETAILER locations. 3D = Product (new TM)
CtExp =  repeat(CtExp',1,T,G) # Columns turn to time

# Cost is high Lag time is low (1)
LagExp = [5 5]; # Rows = TM Columns = Retailers

# Expedited Capacity (to prevent the eval model from just expediting
# everything)
CtExpCapacity = zeros(size(CtExp));
CtExpCapacity[:,:,:] .= 0;

## Expedited transport cost from Supplier to Supplier. ONLY USED IN EVAL MILP.
CtransferExp = [0 25 25; 25 0 25; 25 25 0]; # square matrix of ALL suppliers. cost of lateral transfer between suppliers
CtransferExp = repeat(CtransferExp,1,T,G); 

#valid_transferExprte = [0 1 1;1 0 1; 1 1 0];
valid_transferExprte = [0 0 0;0 0 0; 0 0 0];

valid_transferExprte = repeat(valid_transferExprte*Inf,1,T,G);
valid_transferExprte[isnan.(valid_transferExprte)].=0; # replace NaNs with 0

LagTransferExp = 1; # time lag to transfer between suppliers) ****ASSUME ALL LAGS ARE 1***

# The transferred product still flows to the customer in the same manner as
# normal product. I just break it out seperatley for ease of visualization
# and applying the transfer time lag.
CtransferExpFLOW = CtSexpanded


## Lag Multiplier (to account for Supplier lag in shipping to SLs, not a problem for DCs as current problem stands, ie one TM)
LagSMult = ones(nsup,T,G,ncust)
for i=1:nsup
    for j=1:ncust
        for g = 1:G
            for t = 1:LagS[i,j]
                LagSMult[i, t, g, j] = 0
            end
        end
    end
end
LagSMult

## AM stuff must incoporate both suppliers and customers because both locations have AM
AM_fixed_cost = [121*60; 121*60; 121*60; 121*60; 121*60]; # Suppliers must come first

# ***Suppliers and customers. Suppliers must come first.
K_stable = [1; 1; 1; 1; 1]; # AM Capacity. Rows = Sup and customers. Columns = Time. 3D = Prodcuts
K_stable = repeat(K_stable,1,T,G);
K_stable

# AM Capacity
AM_capacity = 4

###### Straight from Kyle's Code
#
## Network Design Combinations
## Network design for all AM locations (this is a combo of suppliers (DC) and customers (R)
# q = dec2bin(0:2^(nsup+ncust)-1);
# NetDesign = zeros(2^nsup+ncust-1,nsup+ncust);
# for i=1:size(q,1);
#     for j=1:size(q,2);
#         NetDesign(i,j) = str2double(q(i,j)); # List of all network designs. (this is AM locations)
#     end
# end
# NetDesign; # All suppliers are first, customers are last.
# 
# ndesign = size(NetDesign,1); # number of Network Design combinations
# Error check: 2^(nsup+ncust)
# Error check: size(NetDesign,2) == nsup+ncust
# # NetDesign Seperation (JUST FYI)
# 
# NetDesignSup = NetDesign(:,1:nsup)
# NetDesignCust = NetDesign(:,nsup+1:end)
# Set-up for par use
############NetworkDes(ndesign).DesignForceast = [];
############NetworkDes(ndesign).MinPolicy = [];
############NetworkDes(ndesign).MaxPolicy = [];
#### End from Kyle's code

In [27]:
des = size(NetDesign)[1]

yinit = pAll #Start with 10 more products?
MinPolicy = pAll #Min policy (little s)

lbinvenDC = zeros(nsup,T+1,G)
ubinvenDC = zeros(nsup,T+1,G) .+ (1*Inf)
lbinvenSL = zeros(ncust,T+1,G)
ubinvenSL = zeros(nsup, T+1,G) #.+ (Inf)
for g=1:G
    for i=1:nsup
        lbinvenDC[i,1,g] = yinit[i,g]
        lbinvenDC[i,T+1,g] = yinit[i,g]
        ubinvenDC[i,1,g] = yinit[i,g]
        ubinvenDC[i,T+1,g] = yinit[i,g]
    end
    for i=1:ncust
        lbinvenSL[i,1,g] = forecast[i,1,g]
        ubinvenSL[i,1,g] = forecast[i,1,g]
    end
end

In [35]:
CpTM

3×60 Array{Any,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1

In [30]:
#Store the TC and big S policies
TC_stored = zeros(32)
Big_S = zeros(3,3,32)
tick()
# Iterate through network designs and save the policies at the end
@distributed for d=1:des
    AM_force = NetDesign[d,:] #Changes a network design each time (AM locations)
    AMcapability = convert(Int64,sum(AM_force))
    K = K_stable .* AM_force # Eliminates capacity of non-AM locs. (Turn AM on/off)

    model = Model(Gurobi.Optimizer)
    ## Set up Decision Variables
    @variable(model, 0 <= TMprod[g=1:G,t=1:T],Int) #quantity of part g traditionally produced in period t at loc i in P
    @variable(model, 0 <= AMprod[i=1:(nsup+ncust),t=1:T,g=1:G],Int) #quantity of part g produced with AM in period t at location i at DC or Service Loc
    @variable(model, 0 <= transTM[i=1:nsup,t=1:T,g=1:G] <= valid_TMrte[i,t,g],Int) #flow of part g transported from location TM to DC in time t
    @variable(model, 0 <= transDC[i=1:nsup,t=1:T,g=1:G,m=1:ncust] <= valid_Suprte[i,t,g,m],Int) #flow of part g from m DC loc to i SL loc at time t.
    @variable(model, lbinvenDC[i,t,g] <= invenDC[i=1:nsup,t=1:T+1,g=1:G] <= ubinvenDC[i,t,g],Int) #quantity of part g carried as inventory at location i in DC from period t-1 to t
    @variable(model, lbinvenSL[i,t,g] <= invenSL[i=1:ncust,t=1:T+1,g=1:G] <= ubinvenSL[i,t,g],Int) #quantity of part g carried as inventory at location i at SLs from period t
    @variable(model, 0 <= backorder[i=1:ncust,t=1:T+1,g=1:G],Int) #quantity of unsatisfied demand of part g at location i in SL from period t
    @variable(model, 0 <= reorderQTY[i=1:nsup,t=1:T,g=1:G],Int) #quantity to reorder for supplier i at time t for product k
    @variable(model, 0 <= delivered[i=1:ncust,t=1:T,g=1:G],Int) #quantity of item g "delivered/used" at SL i at time t
    @variable(model, p[i=1:nsup,t=1:T,g=1:G], Bin) # reordrer indicator for part g at location i (DC) at time period t
    @variable(model, e[i=1:(ncust+nsup),t=1:T,g=1:G], Bin) #AM production indicator for part g at location i (DCs or SLs) in time period t
    @variable(model, MinPolicy[i,g] <= MaxPolicy[i=1:nsup, g=1:G] <= Inf,Int) #Solving for this, Big S
    #Set up obejctive function
    @objective(model, Min, 
        sum(CpTM[g,t]*TMprod[g,t] for g=1:G,t=1:T) + #TM production cost
        sum(CtTM[i]*transTM[i,t,g] for i=1:nsup, t=1:T, g=1:G) +  #transportation cost TM to supplier
        sum(CtS[i,m]*transDC[i,t,g,m] for i=1:nsup, t=1:T, g=1:G, m=1:ncust) + #transportation supplier to SLs
        sum(Ci[i,t,g]*invenDC[i,t,g] for i=1:nsup, t=2:T+1, g=1:G) + #DCs inventory cost
        sum(CiRetailer[i,t,g]*invenSL[i,t,g] for i=1:ncust, t=2:T+1, g=1:G) + #Customer inventory cost
        sum(CpAM[i,t,g]*AM_fixed_cost[i]*AMprod[i,t,g] for i=1:(nsup+ncust), t=1:T, g=1:G) + #AM production cost
        sum(CBO[i,t,g]*backorder[i,t,g] for i=1:ncust, t=1:T+1, g=1:G)); #backorder cost

    ###Constraints
    #Flow balance produced at time T = ∑ of all things transported at time T
    #i.e. No inventory stored at TM
    for t = 1:T
        for g = 1:G
            @constraint(model, TMprod[g,t] == sum(transTM[i,t,g] for i=1:nsup)) 
        end
    end

    #Inventory Balance at DCs
    for i=1:nsup
        for g=1:G
            for t=1:LagTM[i]
                @constraint(model, invenDC[i,t+1,g] == invenDC[i,t,g] + AMprod[i,t,g] - sum(transDC[i,t,g,m] for m=1:ncust))
            end
            for t=(LagTM[i]+1):T
                @constraint(model, invenDC[i,t+1,g] == invenDC[i,t,g] + AMprod[i,t,g] + sum(transTM[i,t-LagTM[i],g]) - sum(transDC[i,t,g,m] for m=1:ncust))
            end
        end
    end

    #Inventory Balance at SLs
    for i=1:ncust
        for g=1:G
            for t=1:T
                @constraint(model,invenSL[i,t+1,g] == invenSL[i,t,g] + AMprod[i+nsup,t,g] + 
                    sum(LagSMult[j,t,g,i] * transDC[j,maximum([t-LagS[j,i],1]),g,i] for j=1:nsup) - delivered[i,t,g])
            end
        end
    end

    #Backorder
    for i=1:ncust
        for t=1:T
            for g=1:G
                @constraint(model,backorder[i,t+1,g] == backorder[i,t,g] + forecast[i,t,g] - delivered[i,t,g])
            end
        end
    end

    #Min and Max Policy at DCs
    for i=1:nsup
        for g=1:G
            @constraint(model, MinPolicy[i,g] <= invenDC[i,2,g] - (100000 * (1-p[i,1,g])))
            @constraint(model, MaxPolicy[i,g] >= invenDC[i,2,g] - (100000 * (1-p[i,1,g])))
            for t=2:T
                @constraint(model, MinPolicy[i,g] <= invenDC[i,t+1,g] - (100000 * (1-p[i,t,g])) + sum(reorderQTY[i,m,g] for m=(t-1):maximum([t-LagTM[i],1])))
                @constraint(model, MaxPolicy[i,g] >= invenDC[i,t+1,g] - (100000 * (1-p[i,t,g])) + sum(reorderQTY[i,m,g] for m=(t-1):maximum([t-LagTM[i],1])))
            end
        end
    end

    ##### Reorder Constraints
    for i=1:nsup
        for t=1:T
            for g=1:G
                #Reorder quantity
                @constraint(model, reorderQTY[i,t,g] == (MaxPolicy[i,g] - invenDC[i,t+1,g]) * p[i,t,g])

                #Replenishment constraints
                @constraint(model, reorderQTY[i,t,g] <= 100000 * p[i,t,g])
                @constraint(model, reorderQTY[i,t,g] <= MaxPolicy[i,g] - invenDC[i,t+1,g])
                @constraint(model, reorderQTY[i,t,g] >= MaxPolicy[i,g] - invenDC[i,t+1,g] - 10000*(1-p[i,t,g]))
            end
        end
    end

    #Linking Reorder to trans from TM to DCs
    for i=1:nsup
        for t=1:T
            for g=1:G
                @constraint(model, transTM[i,t,g] == reorderQTY[i,t,g])
            end
        end
    end

    #AM capacity and production indicator
    for i=1:(nsup+ncust)
        for t=1:T
            for g=1:G
                @constraint(model, AMprod[i,t,g] <= K[i,t,g] * AM_capacity * e[i,t,g])
            end
            #link e with as a production indicator (wheather or not AM is currently being used for a product)
            @constraint(model, sum(e[i,t,g] for g=1:G) <= 1)
        end
    end

    set_optimizer_attribute(model, "OutputFlag", 1)
    set_optimizer_attribute(model, "MIPGap", .05)
    #set_optimizer_attribute(model, "TimeLimit", )
    optimize!(model);
    TC_stored[d] = objective_value(model)

    Big_S[:,:,d] = value.(MaxPolicy)
end
tock()

┌ Info:  started timer at: 2021-06-16T12:59:42.419
└ @ TickTock /Users/Matthew/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:          0.126622471s: 126 milliseconds
└ @ TickTock /Users/Matthew/.julia/packages/TickTock/RsTHR/src/TickTock.jl:39


Academic license - for non-commercial use only - expires 2021-07-26
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5880 rows, 6330 columns and 17931 nonzeros
Model fingerprint: 0x5bfbcd0e
Model has 540 quadratic constraints
Variable types: 0 continuous, 6330 integer (1440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 7e+03]
  Bounds range     [3e+00, 2e+01]
  RHS range        [1e+00, 1e+05]
Presolve removed 3234 rows and 3999 columns
Presolve time: 0.36s
Presolved: 5922 rows, 3267 columns, 16335 nonzeros
Variable types: 0 continuous, 3267 integer (468 binary)

Root relaxation: objective 4.762888e+05, 1260 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [15]:
#### Eval part
#nd = 16
#initialize all stored variables for later
stored_transDC = zeros(nsup,T,G,ncust,nreal,ndesign)
stored_AMprod = zeros(nsup+ncust,T,G,nreal,ndesign)
stored_SupTransCost = zeros(T,nreal,ndesign)
stored_AMprodcost = zeros(T,nreal,ndesign)
stored_TMprodCost = zeros(T,nreal,ndesign)
stored_InvCarCost = zeros(T,nreal,ndesign)
stored_TC = zeros(T,nreal,ndesign)
stored_reorder_qty = zeros(nsup,T,G,nreal,ndesign)
stored_TMtransCost_Itemized = zeros(nsup,T,nreal,ndesign)
stored_delivered = zeros(ncust,T,G,nreal,ndesign)
#stored_reorder = zeros()
#stored_reorder_delivered = zeros()
stored_TMtransCost = zeros(T,nreal,ndesign)
stored_BOtotal = zeros(ncust,T+1,G,nreal,ndesign)
stored_BOtrue = zeros(ncust, T+1, G, nreal,ndesign)
stored_BOlag = zeros(ncust,T+1,G,nreal,ndesign)
@distributed for nd = 1:ndesign
    #update AM force
    AM_force = NetDesign[nd,:] #changes the network design of AM lcoations
    
    #Setting Min/Max Policy found in MILP
    MaxPolicy = Big_S[:,:,nd]
    MaxPolicy_3d = permutedims(reshape(MaxPolicy,(3,1,3)),[3,2,1]) #changes to be rows=nsup, cols=t, 3d=product
    MinPolicy_3d = permutedims(reshape(MinPolicy,(3,1,3)),[3,2,1])
    
    
    #Loop for demand realizations
    for dr = 1:nreal
        dem_running = dem_real[dr,:,:,:]
        #needs to be reset every run 
        #starting with 5 more products than little s
        yinit = MinPolicy .+ 5
    
        sup = zeros(nsup,T+maximum(LagTM),G)
        #setup so sup is nsup, time, by nprods matrix,
        for i = 1:nsup
            for g = 1:G
                sup[i,1,g] = yinit[i,g]
            end
        end
        
        
        
        previous_orders = zeros(nsup,T+maximum(LagTM),G) # reset for each realization
        reorder_delivered = zeros(nsup,T+maximum(maximum(LagTM)),G) # Destination  matrix for when the re-ordered product will be delivered. Rows = Suppliers. Columns = Time
    
        
        #iterate through time
        for t=1:T
                   
            ## Reduce dimensions of select variables for evaluation in MILP
            # Assumption CtS, CBO, CpAM, K do not change over time. If they do then
            # just need to extract the appropriate columns on the appropriate loop
            # through time.
            dem = dem_running[t,:,:];# extract the current time period for use in MILP
            Ci2 = Ci[:,t:t+1,:]; # reduce to 2
            Ci2[:,1,:] .= 0 # First time period (yinit = 0 cost)
            CBO2 = CBO[:,t:t+1,:]; # reduce to 2
            CBO2[:,1,:] .= 0;
        
            CpAM2 = CpAM[:,t,:] # reduce to 1
        
            # Reduce to 1
            # Eliminate Capacity of Non-AM 'forced' Sites
            K = K_stable[:,t,:].*AM_force;
        
            cdel2 = cdel[:,t,:]; # reduce to 1 * dont have to do t

            CiRetailer2 = CiRetailer[:,t:t+1,:]; # reduce to 2
            CiRetailer2[:,1,:] .= 0;
            
            ####### think i need to store this as a variable later
            #e2 = e[:,t,:]; # reduce to 1 *Dont have to do t
        


            # Not used in MILP but used in future calcs
            CpTM2 = CpTM[:,t,:];

            # Reduce Expedited TM-Cust
            CpExp2 = CpExp[:,t,:]; # reduce to 1
            CtExp2 = CtExp[:,t,:]; # reduce to 1

            CtExpCapacity2 = CtExpCapacity[:,t,:];

        
            # Reduce the valid rte UBs for use in MILP
            valid_Suprte2 = valid_Suprte[:,t,:,:]; # Used in MILP
            valid_TMrte2 = valid_TMrte[:,t,:]; # Not used in this code..
        
        
            # Reduce Sup-Sup Exp transfer variables and UBs
            CtransferExp2 = CtransferExp[:,((t-1)*3+1):(t-1)*3+3,:];
            valid_transferExprte2 = valid_transferExprte[:,((t-1)*3+1):(t-1)*3+3,:];

            CtransferExpFLOW2 = CtransferExpFLOW[:,t:t+1,:];

            yinit = sup[:,t,:] # update inventory for current t
            lbSupply = zeros(nsup,2,G)
            ubSupply = ones(nsup,2,G) .* Inf
            for i = nsup
                for g=1:G
                    lbSupply[i,1,g] = yinit[i,g]
                    ubSupply[i,1,g] = yinit[i,g]
                end
            end
            #yinit = permute(yinit,[2 1 3]); 
            global y = zeros(nsup,T,G)
            

            model = Model(Gurobi.Optimizer);
            ## Set up Decision Variables
             #mp = buildMILP(Ci,CpAM,CtS,cdel,CiRetailer,CBO,e,CpExp,CtExp,CtransferExp,CtransferExpFLOW)
            @variable(model, lbSupply[ns,tt,g] <= invenDC[ns=1:nsup,tt=1:2,g=1:G] <= ubSupply[ns,tt,g]) #inventory quantity of prod g at DC nc at time t
            @variable(model, 0 <= AMprod[i=1:(nsup+ncust),g=1:G] <= K[i,g]) #quantity of product g produced at location i
            #@variable(model, 0 <= transTM[i=1:nsup,g=1:G]) 
            @variable(model, 0 <= transDC[ns=1:nsup,g=1:G,nc=1:ncust] <= valid_Suprte2[ns,g,nc]) #quantity of product g transported from DC ns to SL nc
            @variable(model, 0 <= delivered[nc=1:ncust, g=1:G]) #Quantity of product g delivered/utilized at SL nc
            @variable(model, 0 <= invenSL[nc=1:ncust,tt=1:2,g=1:G] <= 0) #Inventory at SL
            @variable(model, 0 <=backorder[nc=1:ncust,tt=1:2,g=1:G]) #Backorder DV 
            @variable(model, e[i=1:(ncust+nsup),g=1:G], Bin) #AM production indicator
            #@variable(model, 0 <= ExpProd[i=1:ncust,g=1:G]) #Expedited Production
            #@variable(model, 0 <= transExp[i=1:ncust,g=1:G]) #Expedited transportation
            #variable(model, transferExp[]) #Lateral Transfer between DCs
            
            #Set up obejctive function
            @objective(model, Min, 
                sum(Ci2[ns,tt,g] * invenDC[ns,tt,g] for ns=1:nsup, tt=1:2, g=1:G) + #DC inventory cost
                sum(CpAM2[i,g] * AMprod[i,g] for i=1:(nsup+ncust), g=1:G) + #AM production cost
                sum(CtS[ns,nc] * transDC[ns,g,nc] for ns=1:nsup, g=1:G, nc=1:ncust) + #DC to SL transportation cost
                sum(CiRetailer2[nc,tt,g] * invenSL[nc,tt,g] for nc=1:ncust, tt=1:2, g=1:G) + #SL inventory cost
                sum(CBO2[nc,tt,g] * backorder[nc,tt,g] for nc=1:ncust, tt=1:2, g=1:G)) #+ #Backorder Cost
                #sum(CpExp2[nc,g] * ExpProd[nc,g] for nc=1:ncust, g=1:G) + #Expedited production cost
                #sum(CtransferExp2[nc,g] * transExp[nc,g] for nc=1:ncust, g=1:G) + #Expedtied transportation cost
            

            for g = 1:G
                for tt=1:1
                    # Expedited TM Flow Balance
                    # Amount Exp Produced - sum of Amount EXPEDITED out to all RETAILERS.
                    #@constraint(model, ExpProd[g] == sum(transEXP[i,g] for i=1:nsup))
            
                    for ns = 1:nsup

                        # Distribution Center (Supplier) Flow Balance
                        # In from TM, + Current Inv + AM Produced - Leftover Inv - Sum of everything shipped out. 
                        @constraint(model, invenDC[ns,tt,g] + AMprod[ns,g] - invenDC[ns,tt+1,g] - sum(transDC[ns,g,nc] for nc=1:ncust) == 0 )
                        # *** want to add: in all in 1st column, all rows and all out: 1st row all columns
                
                
                        # Build the transfer connections for 'In and out'. It is OK if the diagnoal is 'wrong' bc the UB on it is zero so nothing will ever flow over it.       
                        #zz = zeros(size(CtransferExp));
                        #zz[:,ns,g] = 1;
                        #zz[ns,:,g] = -1;
                        #mp.addcstr({[1 -1],{ns,[tt tt+1],g}},{ns,tt,g},{-1,{ns,':',g}},0,0,0,0,0,0,{zz,{':',':',g}},{-1,{ns,':',g}},'=',0);

                        #mp.addcstr(0,0,0,0,0,0,0,0,0,{':',ns,g},'=',{ns,':',g}); # Everything in from a lateral transfer must be transported across the special transfer arcs.


                    end

                    for nc = 1:ncust;
                        # Retailer Flow Balance
                        # Sum of everything in from all suppliers - delivered +Inv In - Inv out
                            # This places a '1' in the appropraite time period that product left the supplier based on the lag.
                        @constraint(model, AMprod[nc+nsup,g] + sum(transDC[ns,g,nc] for ns=1:nsup) - delivered[nc,g] + invenSL[nc,tt,g] - invenSL[nc,tt+1,g] == 0)

                        #BO
                        # in from delivered, -BO in + BO out, = Demand
                        @constraint(model, delivered[nc,g] - backorder[nc,tt,g] + backorder[nc,tt+1,g]== dem[nc, g])
                    end

                    # Production indicator constraint and AM capacity enforecement
                    #  Fixed Cost no longer active in MILP.
                    # ****************indedxing is different becuase it is for both
                    # suppliers and customers.**********
                    for i = 1:nsup+ncust;
                        @constraint(model, AMprod[i,g] <= K[i,g] * e[i,g])
                    end
            end #end of tt loop
            end #end of G loop
            #link e with as a production indicator (wheather or not AM is currently being used for a product)
            for i = 1:(nsup+ncust)
                @constraint(model, sum(e[i,g] for g=1:G) <= 1)
            end
            set_optimizer_attribute(model, "OutputFlag", 1)
            #set_optimizer_attribute(model, "MIPGap", .001)
            #set_optimizer_attribute(model, "TimeLimit", )
            optimize!(model);
            
            #store the data 
            stored_transDC[:,t,:,:,dr,nd] .= value.(transDC)
            stored_AMprod[:,t,:,dr,nd] .= value.(AMprod)
            stored_SupTransCost[t,dr,nd] = sum(CtS[ns,nc] * value.(transDC)[ns,g,nc] for ns=1:nsup, g=1:G, nc=1:ncust)
            stored_AMprodcost[t,dr,nd] = sum(CpAM2[i,g] * value.(AMprod)[i,g] for i=1:(nsup+ncust), g=1:G)
            stored_InvCarCost[t,dr,nd] = sum(Ci2[ns,tt,g] * value.(invenDC)[ns,tt,g] for ns=1:nsup, tt=1:2, g=1:G)
            stored_TC[t,dr,nd] = objective_value(model)
            
            
            #Update inventory for next time period
            sup[:,t+1,:] = value.(invenDC)[:,2,:] + sup[:,t+1,:] # Represents the amount of inventory on hand to START that period.  Row = Suppliers. Columns = Time. 
            
            ######MIGHT BE OF INTEREST LATER NEED TO BUILD STRUCTURE FOR THIS STORAGE
            ## AM locations used
            #AM_loc_used(:,t,:) = x.CpAM > 0;
            ## AM locations used
            #AM_qty_used(:,t,:) = x.CpAM;
        
            #Inventory Carried
            y[:,t,:] = value.(invenDC)[:,2,:] # Inventory carried between periods. (t1 is the inventory that was carried from t1 to t2)
                            # This is different than 'sup' at the end of the loop becuase y extracts your inventory before re-orders arrive.
                            # Use y for inventory carried between periods.
                            # Use sup to see what the actual inventory is at the start of a period (it adds in re-orders)
            
            # Determine the re-order quantity and place the re-ordered quantiy
            # in sup matrix based on the transportation lag from TM to sup.
        
            # This should prevent re-order every period, by looking back over
            # the lag time. If your inventory + whatever you have enroute is <
            # Minpolicy then order again
            log = zeros(nsup,1,G)
            if t==1
                log = reshape((y[:,t,:] .+ previous_orders[:,t,:] .< MinPolicy),(nsup,1,G))
            else 
                log = reshape((y[:,t,:] .+ permutedims(sum(previous_orders[:,maximum([1, t-6]):t-1,:],dims=2),[1,3,2])) .< MinPolicy,(nsup,1,G))
            end
        """
            #if t == 1
            #   log =  y[:,t,:] +  sum(previous_orders(:,t,:),2) < MinPolicy ;
            #elseif t == 2
            #   log =  y(:,t,:] +  sum(previous_orders(:,t-1,:),2) < MinPolicy;
            #elseif t == 3
            #   log =  y(:,t,:] +  sum(previous_orders(:,[t-2:t-1],:),2) < MinPolicy  ;
            #elseif t == 4
            #   log =  y(:,t,:] +  sum(previous_orders(:,[t-3:t-1],:),2) < MinPolicy ;
            #elseif t == 5
            #  log =   y(:,t,:] +  sum(previous_orders(:,[t-4:t-1],:),2) < MinPolicy ;
            #elseif t == 6
            #  log =   y(:,t,:] +  sum(previous_orders(:,[t-5:t-1],:),2) < MinPolicy ;
            #elseif t == 7
            #  log =   y(:,t,:) +  sum(previous_orders(:,[t-6:t-1],:),2) < MinPolicy ;
            #elseif t >= 8
            #   log =  y(:,t,:) +  sum(previous_orders(:,[t-6:t-1],:),2) < MinPolicy ;
            #end
            
        """
            b = log.*MaxPolicy_3d; # Zeros out corresponding policies that dont need to order
            bb = log.*reshape(y[:,t,:],(3,1,3)); # Zeros out corresponding suppliers that dont need to order
            reorder_qty  = b.-bb; # subtract current inventory from policies. This is the re-order qty per supplier.
            previous_orders[:,t,:] = reorder_qty
        
            # Now place in the appropriate place (using nested loop to get it right) ##COMMENT: g loop might not be needed because same lag time
            for ns=1:nsup
                for g=1:G
                    sup[ns,t+LagTM[ns],g] = reorder_qty[ns,1,g] + sup[ns,t+LagTM[ns],g] # Add the re-ordered qty to the location where it will be delivered.
                    reorder_delivered[ns,t+LagTM[ns],:] = reorder_qty[ns,1,:] # Just a data tracking matrix. Displays when re-ordered goods, for  each supplier are  delivered.
                end
            end
                
        
            
            """ From Kyle;s original. Think I covered it with last loop
            #aa = repeat(1:G,nsup,1); # this is the dimensions that will be used in the sub2ind function.
            #ii = sub2ind(size(sup),repeat(1:nsup,1,G),repeat(t.+LagTM,1,G),aa[:]') ;# find the location that the ordered goods will arrive and place them in sup
                                                                                # I,J,G
            #sup[ii] =  reorder_qty[:]' + sup[ii]; # Add the re-ordered qty to the location where it will be delivered.
            """
    
            # Save Data  
            stored_reorder_qty[:,t,:,dr,nd] = reorder_qty ; # Reordered quantity per customer per product
    
            
            """ NOT SURE IF NEEDED
        
            # NOTE: Sup matrix recieved the re-ordered goods.  At this point,     ################ WORK ON
            # it represents the full amount of inventory available at
            # suppliers for t+1 and on.
            time(t).sup = sup; # This is the inventory available at the beginiing of the next time period time period.
                          # ex. time(1).sup represents the supply conditions to start t2.
                          # NOTE: AM Capacity is added in at the beginning
                          # of next period.
            """
    
            #Costs associated with re-orders
            
            # Production Cost TM
            stored_TMprodCost[t,dr,nd] = sum(reorder_qty.*CpTM2); # Amount re-orderd, per supplier * the associated production cost
            
            # Transportation Cost TM - Supplier
            TMtransCost = reorder_delivered[:,t,:].*CtTM;  # TM Trans Cost per time period to ship product to suppliers: Itemized out to see when costs are incurred. Rows = Suppliers Columns = Time.

            stored_TMtransCost_Itemized[:,t,dr,nd] = sum(TMtransCost,dims=2); # Tracks Transportation cost from TM to suppliers.  Itemized by supplier. Assume charge for transportation is made at the time of the order (current t), not when the items are delivered (like above variable)
                        #### CHECK IF THIS IS THE RIGHT DIMENSION
        
            stored_TMtransCost[t,dr,nd] = sum(sum(sum(TMtransCost,dims=2))); # Total TM trans cost for all suppliers in a single time period
            #NOTE: maybe delete on of the variables above. Or save all but last one in an auxiallary location.
        
             
            # Delivered Product 
        
            # Take the supplier - customer flow, reference the transportation lag for each
            # connection. Output, how much product delivered in each time period.
            # Generate a delivered matrix for each supplier that displays how much product is deliverd to each customer and in what time period.
            # determine maximum lag in the network
            temp1 = maximum(LagS);
            temp2 = maximum(LagTM);
            LagMax = maximum([temp1 temp2])
            #temp3 = maximum(LagTransferExp);
            #temp4 = maximum(LagExp);
            #LagMax = maximum([temp1 temp2 temp3 temp4]);
            
        
            ###### WORK ON
            #b = zeros(ncust,T+LagMax+LagTransferExp,G); 
            b = zeros(ncust, T+LagMax, G) # Destination matrix for delivered products. Rows = Cust Columns = Time Periods. Generate for all time periods plus largest possible lag if order was made in the last period.
                                          # Generate matrix with all customers and the total # of time periods of lag so everything combines nicely in the end
            
            #### NOTE: This can be done better with sums
            #Execute for Suppliers-Customers
            for i = 1:nsup;
                #a = zeros(ncust,T+LagMax+LagTransferExp,G); 
                a = zeros(ncust,T+LagMax,G)# Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
                for g = 1:G;
                    for j = 1:ncust;
                        a[j,t+LagS[i,j],g] = value.(transDC)[i,g,j]; # 'a' is a single suppliers delivery 'windows' to all customers. 
                    end                             # Logic: Assign the flow from S1 to C1, put it in a matrix row for cust 1 and apply the time lag to when  it is delivered (column)
                end
                b = b .+ a; # Delivered matrix. Rows = Customers. Columns = time periods of delivery .
            end           # Combine all supplier delivery matricies into one. Final b matrix displays when the goods, that are shipped out from all suppliers in time(t), are recieved by the customers.
            
    
        """   LATERAL TRANSFER, BRING IN LATER
              # Execute for Sup-Sup Lateral Transfers then to customers
              # Only change is we apply a transfer lag to the transferred product
              # only
        for i = 1:nsup;
            a = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
            for g = 1:G;
                for j = 1:ncust;
                    a(j,t+LagS(i,j)+LagTransferExp,g) = x.CtransferExpFLOW(i,j,g); # 'a' is a signle suppliers delivery 'windows' to all customers. 
                end                             # Logic: Assign the flow from S1 to C1, put it in a matrix row for cust 1 and apply the time lag to when  it is delivered (column)
            end
            b = b + a ;# Delivered matrix. Rows = Customers. Columns = time periods of delivery .
        end
        """
        """ EXPEDITED SHIPPING
        # Execute for Expedited product (TM-Customer(RETAILER) direct
        for i = 1:nTM; #Should be 1x TM per product. nTM =2 would be 2x TMs per procut. KEEP AT 1
            a = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
            for g = 1:G;
                for j = 1:ncust;
                    a(j,t+LagExp(i,j),g) = x.CtExp(j,i,g);
                end
            end
            b = b + a;
        end
        """

        
            stored_delivered[:,t,:,dr,nd] = b[:,t,:]; # Delivered Matrix. Displays when (in terms of time) product that was shipped out in time (t) is recieved by customers. Rows = Customers Columns = Time Periods.
                        # The sum of all product in time(1).del is theamount of goods that was shipped out in time period 1.
        
            # Determine Backorder and Update Demand
        
            # Backorder
            stored_BOtotal[:,t,:,dr,nd] = dem .- stored_delivered[:,t,:,dr,nd]; # # Backorder for current time period. Rows = Customer Columns = time periods. Subtract goods delivered in current time period from demand in current time period. This is the amount of product that the customer still doesnt have, do to lag and actual shortage 
            stored_BOtrue[:,t,:,dr,nd] = value.(backorder)[:,2,:] # Rows: Customers. dimension = products. True backorder is an actual shortage of product
            stored_BOlag[:,t,:,dr,nd] =  stored_BOtotal[:,t,:,dr,nd] - stored_BOtrue[:,t,:,dr,nd] ;     
            # Want to distinguish between a time lag backorder and a shortage backorder.
            # BO is the amount of product a customer is short going from t to
            # t+1. Includes both true and time lag product
            # Time lag backorder: Inventory was on hand, it was shipped out immediatley, it just take t+x time periods for it to be delivered to customer.
            # True Shortage backorder: Not enough inventory in that time period to ship out.


            # Update demand for next period by adding the true shortage
                if t < T # this just takes care of the edge case of the  last period. No change  to what it does.
                    dem_running[t+1,:,:] = dem_running[t+1,:,:] + stored_BOtrue[:,t,:,dr,nd];
                else
                    #dem_running[t+1,:,:] = stored_BOtrue[:,t,:,dr,nd];
                end
            # No need to update demand becuase backorder shortage already spoken for. It is enroute, no need to see it in the demand matrix. 
        
        end #end of loop through t=1:60
    
    end #end of demand realizations
end #end of design loop

Academic license - for non-commercial use only - expires 2021-07-26
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 96 columns and 135 nonzeros
Model fingerprint: 0x1a35030b
Variable types: 81 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+02]
  Bounds range     [2e+01, 2e+01]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 6117.0000000
Presolve removed 41 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 460 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.600000000000e+02, best bound 4.600000000000e+02, gap 0.0000%

User-callback calls 41, time in user-callback 0.00 sec
Academic license - for non-commercial u

Task (done) @0x0000000119835210

In [76]:
sum(mean(stored_TC,dims=2),dims=1)

1×1×32 Array{Float64,3}:
[:, :, 1] =
 30273.600000000006

[:, :, 2] =
 28268.36

[:, :, 3] =
 28835.040000000005

...

[:, :, 30] =
 27958.760000000002

[:, :, 31] =
 28351.199999999997

[:, :, 32] =
 26520.200000000004

In [80]:
stored_reorder_qty

3×60×3×50×32 Array{Float64,5}:
[:, :, 1, 1, 1] =
 32.0  0.0   0.0  0.0  0.0  0.0  0.0  …  0.0  0.0   0.0  32.0  0.0  0.0  0.0
 33.0  0.0   0.0  0.0  0.0  0.0  0.0     0.0  0.0   0.0  33.0  0.0  0.0  0.0
  0.0  0.0  25.0  0.0  0.0  0.0  0.0     0.0  0.0  20.0   0.0  0.0  0.0  0.0

[:, :, 2, 1, 1] =
 18.0  18.0  0.0  0.0  0.0  0.0   0.0  …  0.0   0.0  18.0  18.0  0.0  0.0
 19.0   0.0  0.0  0.0  0.0  0.0   0.0     0.0   0.0  19.0   0.0  0.0  0.0
  0.0   2.0  5.0  9.0  0.0  0.0  18.0     0.0  17.0   0.0   0.0  0.0  0.0

[:, :, 3, 1, 1] =
 31.0   0.0  0.0  0.0  0.0  0.0  0.0  …   0.0  0.0  0.0  31.0  0.0  0.0  0.0
 32.0   0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  32.0  0.0  0.0  0.0
  0.0  19.0  0.0  0.0  0.0  0.0  0.0     21.0  0.0  0.0   0.0  0.0  0.0  0.0

[:, :, 1, 2, 1] =
 32.0   0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  32.0  0.0  0.0   0.0
 33.0   0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  33.0  0.0  0.0   0.0
  0.0  24.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  

In [70]:
mean(sum(sum(sum(stored_BOtotal,dims=1),dims=2),dims=3),dims=4)

1×1×1×1×32 Array{Float64,5}:
[:, :, 1, 1, 1] =
 1077.5

[:, :, 1, 1, 2] =
 1077.5

[:, :, 1, 1, 3] =
 1077.5

...

[:, :, 1, 1, 30] =
 1077.5

[:, :, 1, 1, 31] =
 1077.5

[:, :, 1, 1, 32] =
 1077.5

In [75]:
sum(sum(sum(stored_AMprod,dims=1),dims=2),dims=3)

1×1×1×50×32 Array{Float64,5}:
[:, :, 1, 1, 1] =
 0.0

[:, :, 1, 2, 1] =
 0.0

[:, :, 1, 3, 1] =
 0.0

...

[:, :, 1, 48, 1] =
 0.0

[:, :, 1, 49, 1] =
 0.0

[:, :, 1, 50, 1] =
 0.0

[:, :, 1, 1, 2] =
 60.0

[:, :, 1, 2, 2] =
 60.0

[:, :, 1, 3, 2] =
 60.0

...

[:, :, 1, 48, 2] =
 60.0

[:, :, 1, 49, 2] =
 60.0

[:, :, 1, 50, 2] =
 60.0

[:, :, 1, 1, 3] =
 60.0

[:, :, 1, 2, 3] =
 60.0

[:, :, 1, 3, 3] =
 60.0

...

[:, :, 1, 48, 3] =
 60.0

[:, :, 1, 49, 3] =
 60.0

[:, :, 1, 50, 3] =
 60.0

...

[:, :, 1, 1, 30] =
 69.0

[:, :, 1, 2, 30] =
 68.0

[:, :, 1, 3, 30] =
 72.0

...

[:, :, 1, 48, 30] =
 72.0

[:, :, 1, 49, 30] =
 78.0

[:, :, 1, 50, 30] =
 71.0

[:, :, 1, 1, 31] =
 74.0

[:, :, 1, 2, 31] =
 83.0

[:, :, 1, 3, 31] =
 78.0

...

[:, :, 1, 48, 31] =
 79.0

[:, :, 1, 49, 31] =
 81.0

[:, :, 1, 50, 31] =
 86.0

[:, :, 1, 1, 32] =
 129.0

[:, :, 1, 2, 32] =
 128.0

[:, :, 1, 3, 32] =
 132.0

...

[:, :, 1, 48, 32] =
 132.0

[:, :, 1, 49, 32] =
 138.0

[:, :, 1, 50, 32] =
 131.0

In [ ]:
stored_transDC
stored_AMprod
stored_SupTransCost
stored_AMprodcost
stored_TMprodCost 
stored_InvCarCost
stored_TC
stored_reorder_qty
stored_TMtransCost_Itemized
stored_delivered
#stored_reorder = zeros()
#stored_reorder_delivered = zeros()
stored_TMtransCost
stored_BOtotal
stored_BOtrue
stored_BOlag

In [229]:
## Reduce dimensions of select variables for evaluation in MILP
            # Assumption CtS, CBO, CpAM, K do not change over time. If they do then
            # just need to extract the appropriate columns on the appropriate loop
            # through time.
            dem = dem_running[t,:,:];# extract the current time period for use in MILP
            Ci2 = Ci[:,t:t+1,:]; # reduce to 2
            Ci2[:,1,:] .= 0 # First time period (yinit = 0 cost)
            CBO2 = CBO[:,t:t+1,:]; # reduce to 2
            CBO2[:,1,:] .= 0;
        
            CpAM2 = CpAM[:,t,:] # reduce to 1
        
            # Reduce to 1
            # Eliminate Capacity of Non-AM 'forced' Sites
            K = K_stable[:,t,:].*AM_force;
        
            cdel2 = cdel[:,t,:]; # reduce to 1 * dont have to do t

            CiRetailer2 = CiRetailer[:,t:t+1,:]; # reduce to 2
            CiRetailer2[:,1,:] .= 0;
            
            ####### think i need to store this as a variable later
            #e2 = e[:,t,:]; # reduce to 1 *Dont have to do t
        


            # Not used in MILP but used in future calcs
            CpTM2 = CpTM[:,t,:];

            # Reduce Expedited TM-Cust
            CpExp2 = CpExp[:,t,:]; # reduce to 1
            CtExp2 = CtExp[:,t,:]; # reduce to 1

            CtExpCapacity2 = CtExpCapacity[:,t,:];

        
            # Reduce the valid rte UBs for use in MILP
            valid_Suprte2 = valid_Suprte[:,t,:,:]; # Used in MILP
            valid_TMrte2 = valid_TMrte[:,t,:]; # Not used in this code..
        
        
            # Reduce Sup-Sup Exp transfer variables and UBs
            CtransferExp2 = CtransferExp[:,((t-1)*3+1):(t-1)*3+3,:];
            valid_transferExprte2 = valid_transferExprte[:,((t-1)*3+1):(t-1)*3+3,:];

            CtransferExpFLOW2 = CtransferExpFLOW[:,t:t+1,:];

            yinit = sup[:,t,:] # update inventory for current t

LoadError: UndefVarError: sup not defined

In [115]:


## MILP: Evaluate a single period at at time.

 mp = buildMILP(Ci,CpAM,CtS,cdel,CiRetailer,CBO,e,CpExp,CtExp,CtransferExp,CtransferExpFLOW)


# clear mp;
# mp = Milp('PPlan');
# mp.addobj('min',Ci,CpAM,CtS,cdel,CiRetailer,CBO,e,CpExp,CtExp,CtransferExp,CtransferExpFLOW); # Objective
# 1 in -1 out
for g = 1:G;
    for tt=1:1;
        
        
        # Expedited TM Flow Balance
        # Amount Exp Produced - sum of Amount EXPEDITED out to all RETAILERS.
        mp.addcstr(0,0,0,0,0,0,0,{1,{1,tt,g}},{-1,{':',tt,g}},0,0,'=',0); #(C1)

        for ns = 1:nsup;
            # Distribution Center (Supplier) Flow Balance
            # In from TM, + Inv In + AM Produced - Inv Out - Sum of everything shipped out. 
             # *** want to add: in all in 1st column, all rows and all out: 1st row all columns
             # Build the transfer connections for 'In and out'. It is OK if the diagonal is 'wrong' bc the UB on it is zero so nothing will ever flow over it.       
             zz = zeros(size(CtransferExp));
                    zz(:,ns,g) = 1;
                    zz(ns,:,g) = -1;
            mp.addcstr({[1 -1],{ns,[tt tt+1],g}},{ns,tt,g},{-1,{ns,':',g}},0,0,0,0,0,0,{zz,{':',':',g}},{-1,{ns,':',g}},'=',0);
        
            mp.addcstr(0,0,0,0,0,0,0,0,0,{':',ns,g},'=',{ns,':',g}); # Everything in from a lateral transfer must be transported across the special transfer arcs.

        end
        for nc = 1:ncust;
            # Retailer Flow Balance
            # Sum of everything in from all suppliers - delivered +Inv In - Inv out

            mp.addcstr(0,{nc+nsup,tt,g},{':',nc,g},{-1,{nc,tt,g}},{[1 -1],{nc,[tt tt+1],g}},0,0,0,{nc,tt,g},0,{':',nc,g},'=',0); # This places a '1' in the appropraite time period that product left the supplier based on the lag.
            
            #BO
            # in from delivered, -BO in + BO out, = Demand
            mp.addcstr(0,0,0,{nc,tt,g},0,{[-1 1],{nc,[tt tt+1],g}},0,0,0,0,0,'=',dem(nc,tt,g));
        end
              
        
        # Production indicator constraint and AM capacity enforecement
        #  Fixed Cost no longer active in MILP.
           # ****************indedxing is different becuase it is for both
           # suppliers and customers.**********
            for ns = 1:nsup+ncust;
                mp.addcstr(0,{ns,tt,g},0,0,0,0,'<=',{K(ns,tt,g),{ns,tt,g}},0,0,0,0);
                
                mp.addcstr(0,0,0,0,0,0,{ns,tt,':'},0,0,0,0,'=',1);
            end
        
    end

end
# EXPEDITED and TRANSFER TURNED ON/OFF **** with ub
mp.addlb([yinit zeros(1,nsup*tt,G)],0,0,0,0,0,0,0,0,0,0);
#mp.addub([yinit Inf(1,nsup*tt,G)],K,valid_Suprte,Inf,0,[zeros(ncust,1,G) Inf(ncust,tt,G)],1,0,0,0,0);
mp.addub([yinit Inf(1,nsup*tt,G)],K,valid_Suprte,Inf,0,[zeros(ncust,1,G) Inf(ncust,tt,G)],1,Inf,CtExpCapacity,valid_transferExprte,valid_Suprte);
mp.addctype('C','C','C','C','C','C','B','C','C','C','C'); 
## 
##
#mp.dispmodel
## SOLVE USING MATLAB LINPROG
lp = mp.milp2ilp
#options = optimoptions('intlinprog','Display','off');
[x,TC] = intlinprog(lp{:});
x = mp.namesolution(x);
## SOLVE USING GUROBI (alternate method)
# model = mp.milp2gb;
# params.outputflag = 0;
# result = gurobi(model, params);
# x = mp.namesolution(result.x);
# TC = result.objval;
# x;
##
# for g = 1:G # create 2x seperate reports for each product
# mdisp(dem(:,:,g),[],[],['dem' num2str(g)]);
# mdisp(yinit(:,:,g),[],[],['yinint' num2str(g)]);
# mdisp(x.Ci(:,:,g),[],[],['Ci' num2str(g)]);
# mdisp(x.CpAM(:,:,g),[],[],['CpAM' num2str(g)]);
# mdisp(x.e(:,:,g),[],[],['e' num2str(g)]);
# mdisp(x.CtS(:,:,g),[],[],['CtS' num2str(g)]);
# mdisp(x.CpExp(:,:,g),[],[],['CpExp' num2str(g)]);
# mdisp(x.CtExp(:,:,g),[],[],['CtExp' num2str(g)]);
# mdisp(x.CtExp(:,:,g),[],[],['CtExp' num2str(g)]);
# mdisp(x.CtransferExp(:,:,g),[],[],['CtransferExp' num2str(g)]);
# mdisp(x.CtransferExpFLOW(:,:,g),[],[],['CtransferExpFLOW' num2str(g)]);
# mdisp(x.cdel(:,:,g),[],[],['cdel' num2str(g)]);
# mdisp(x.CiRetailer(:,:,g),[],[],['CiRetailer' num2str(g)]);
# mdisp(x.CBO(:,:,g),[],[],['CBO' num2str(g)]);
# end
# TC
## Consoidate Data

  # Save Flow
        time(t).F = x.CtS;
        time(t).AMprod = x.CpAM;
        # Save SupTransCost
        time(t).SupTransCost = sum(sum(sum(x.CtS.*CtS))) + sum(sum(sum(x.CtransferExpFLOW.*CtransferExpFLOW))); # The expedited flow is the product that was transferred between supplies and was delivered to customers. There is no change in how it is deliverd in terms of time and cost.
        time(t).AMProdCost = sum(sum(sum(x.CpAM.*CpAM)));
        time(t).InvCarCost = sum(sum(sum(x.Ci.*Ci)));
        time(t).ExpCp = sum(x.CpExp); # Expedited TM to Cust
        time(t).ExpF = x.CtExp ;# Expedited TM to cust actual flow
        time(t).x = x;
        
        time(t).CtransferExpFLOW = CtransferExpFLOW;
        # Expedited Costs
            time(t).ExpProdCost = sum(sum(x.CpExp.*CpExp)) ;# TM - Cust prod
            time(t).ExpTransCost = sum(sum(x.CtExp.*CtExp)); # TM - Cust actual trans
            
            time(t).ExpTransferCost = sum(sum(sum(x.CtransferExp.*CtransferExp))); # Expedited lateral transfer
            
        # Update Inventory for next time period 
        sup(:,t+1,:) = x.Ci(:,2,:) + sup(:,t+1,:); # Represents the amount of inventory on hand to START that period.  Row = Suppliers. Columns = Time. 
                                         # Re-ordered product not accounted for, yet. It is added into this matrix later in the loop.                          # for, yet. This matrix is
                                        
        # AM locations used
        AM_loc_used(:,t,:) = x.CpAM > 0;
        # AM locations used
        AM_qty_used(:,t,:) = x.CpAM;

             # Inventory Carried
              y(:,t,:) = x.Ci(:,2,:); # Inventory carried between periods. (t1 is the inventory that was carried from t1 to t2)
                            # This is different than 'sup' at the end of the loop becuase y extracts your inventory before re-orders arrive.
                            # Use y for inventory carried between periods.
                            # Use sup to see what the actual inventory is at the start of a period (it adds inre-orders)
        
        # Determine the re-order quantity and place the re-ordered quantiy
        # in sup matrix based on the transportation lag from TM to sup.
        
        # This should prevent re-order every period, by looking back over
        # the lag time. If your inventory + whatever you have enroute is <
        # Minpolicy then order again
        
        if t == 1
           log =  y(:,t,:) +  sum(previous_orders(:,t,:),2) < MinPolicy ;
        elseif t == 2
           log =  y(:,t,:) +  sum(previous_orders(:,t-1,:),2) < MinPolicy;
        elseif t == 3
           log =  y(:,t,:) +  sum(previous_orders(:,[t-2:t-1],:),2) < MinPolicy  ;
        elseif t == 4
           log =  y(:,t,:) +  sum(previous_orders(:,[t-3:t-1],:),2) < MinPolicy ;
        elseif t == 5
          log =   y(:,t,:) +  sum(previous_orders(:,[t-4:t-1],:),2) < MinPolicy ;
        elseif t == 6
          log =   y(:,t,:) +  sum(previous_orders(:,[t-5:t-1],:),2) < MinPolicy ;
        elseif t == 7
          log =   y(:,t,:) +  sum(previous_orders(:,[t-6:t-1],:),2) < MinPolicy ;
        elseif t >= 8
           log =  y(:,t,:) +  sum(previous_orders(:,[t-6:t-1],:),2) < MinPolicy ;
        end
             
        
        #log = y(:,t,:) < MinPolicy; # Identify which suppliers need to re-order which product
        b = log.*MaxPolicy; # Zeros out corresponding policies that dont need to order
        bb = log.*y(:,t,:); # Zeros out corresponding suppliers that dont need to order
        reorder_qty  = b-bb; # subtract current inventory from policies. This is the re-order qty per supplier.
        previous_orders(:,t,:) = reorder_qty
        # Now place in the appropraitae
        aa = repmat(1:G,nsup,1); # this is the dimensions that will be used in the sub2ind function.
        ii = sub2ind(size(sup),repmat(1:nsup,1,G),repmat(t+LagTM,1,G),aa(:)') ;# find the location that the ordered goods will arrive and place them in sup
                                                                                # I,J,G
        sup(ii) =  reorder_qty(:)' + sup(ii); # Add the re-ordered qty to the location where it will be delivered.


        # Save Data
        time(t).reorder_qty = reorder_qty ;# Reordered quantity per customer per product
        time(t).reorder_delivered = zeros(nsup,T+max(max(LagTM)),G); # Destination  matrix for when the re-ordered product will be delivered. Rows = Suppliers. Columns = Time
        time(t).reorder_delivered(ii) = reorder_qty(:)'; # Just a data tracking matrix. Displays when re-ordered goods, for  each supplier are  delivered.
        
        
           
        # NOTE: Sup matrix recieved the re-ordered goods.  At this point,
        # it represents the full amount of inventory available at
        # suppliers for t+1 and on.
        sup;
        time(t).sup = sup; # This is the inventory available at the beginiing of the next time period time period.
                          # ex. time(1).sup represents the supply conditions to start t2.
                          # NOTE: AM Capacity is added in at the beginning
                          # of next period.
      
                          
                            
        # Costs associated with re-orders
        
        # Production Cost TM
        time(t).TMprodCost = sum(sum(reorder_qty.*CpTM)); # Amount re-orderd, per supplier * the associated production cost
         
        # Transportation Cost TM - Supplier
        TMtransCost = time(t).reorder_delivered.*CtTM;  # TM Trans Cost per time period to ship product to suppliers: Itemized out to see when costs are incurred. Rows = Suppliers Columns = Time.
        time(t).TMtransCost_Itemized = sum(TMtransCost,2); # Tracks Transportation cost from TM to suppliers.  Itemized by supplier. Assume charge for transportation is made at the time of the order (current t), not when the items are delivered (like above variable)
        time(t).TMtransCost = sum(sum(sum(TMtransCost,2))); # Total TM trans cost for all suppliers in a single time period
        #NOTE: maybe delete on of the variables above. Or save all but last one in an auxiallary location.
        
        
        
        
         # Delivered Product 
        
        # Take the supplier - customer flow, reference the transportation lag for each
        # connection. Output, how much product delivered in each time period.
        # Generate a delivered matrix for each supplier that displays how much product is deliverd to each customer and in what time period.
       # determine maximum lag in the network
        temp1 = max(LagS,[],'all');
        temp2 = max(LagTM,[],'all');
        temp3 = max(LagTransferExp,[],'all');
        temp4 = max(LagExp,[],'all');
        LagMax = max([temp1 temp2 temp3 temp4]);

        b = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix for delivered products. Rows = Cust Columns = Time Periods. Generate for all time periods plus largest possible lag if order was made in the last period.
                                          # Generate matrix with all customers and the total # oftimeperiods of lagsoeverything combines nicelyin the end
      # Execute for Suppliers-Customers
        for i = 1:nsup;
            a = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
            for g = 1:G;
            for j = 1:ncust;
                a(j,t+LagS(i,j),g) = x.CtS(i,j,g); # 'a' is a signle suppliers delivery 'windows' to all customers. 
            end                             # Logic: Assign the flow from S1 to C1, put it in a matrix row for cust 1 and apply the time lag to when  it is delivered (column)
            end
            b = b + a; # Delivered matrix. Rows = Customers. Columns = time periods of delivery .
        end           # Combine all supplier delivery matricies into one. Final b matrix displays when the goods, that are shipped out from all suppliers in time(t), are recieved by the customers.
       
        
     # Execute for Sup-Sup Lateral Transfers then to customers
     # Only change is we apply a transfer lag to the transferred product
     # only
        for i = 1:nsup;
            a = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
            for g = 1:G;
            for j = 1:ncust;
                a(j,t+LagS(i,j)+LagTransferExp,g) = x.CtransferExpFLOW(i,j,g); # 'a' is a signle suppliers delivery 'windows' to all customers. 
            end                             # Logic: Assign the flow from S1 to C1, put it in a matrix row for cust 1 and apply the time lag to when  it is delivered (column)
            end
            b = b + a ;# Delivered matrix. Rows = Customers. Columns = time periods of delivery .
        end      
        
        
        
        
        # Execute for Expedited product (TM-Customer(RETAILER) direct
        for i = 1:nTM; #Should be 1x TM per product. nTM =2 would be 2x TMs per procut. KEEP AT 1
            a = zeros(ncust,T+LagMax+LagTransferExp,G); # Destination matrix same size as b for each supplier and observe when that suppliers goods are deliverd to the customers.
            for g = 1:G;
            for j = 1:ncust;
                a(j,t+LagExp(i,j),g) = x.CtExp(j,i,g);
            end
            end
             b = b + a;
        end
        
        time(t).del = b; # Delivered Matrix. Displays when (in terms of time) product that was shipped out in time (t) is recieved by customers. Rows = Customers Columns = Time Periods.
                        # The sum of all product in time(1).del is theamount of goods that was shipped out in time period 1.
        
    
                        
          # Determine Backorder and Update Demand
        
        # Backorder
        BOtotal(:,t,:) = dem - time(t).del(:,t,:); # # Backorder for current time period. Rows = Customer Columns = time periods. Subtract goods delivered in current time period from demand in current time period. This is the amount of product that the customer still doesnt have, do to lag and actual shortage 
        BOtrue(:,t,:) = x.CBO(:,2,:); # Rows: Customers. dimension = products. True backorder is an actual shortage of product
        BOlag(:,t,:) =  BOtotal(:,t,:) - BOtrue(:,t,:) ;     
        # Want to distinguish between a time lag backorder and a shortage backorder.
        # BO is the amount of product a customer is short going from t to
        # t+1. Includes both true and time lag product
        # Time lag backorder: Inventory was on hand, it was shipped out immediatley, it just take t+x time periods for it to be delivered to customer.
        # True Shortage backorder: Not enough inventory in that time period to ship out.
        
        
        # Update demand for next period by adding the true shortage
            if t < T # this just takes care of the edge case of the  last period. No change  to what it does.
            dem_running(:,t+1,:) = dem_running(:,t+1,:) + BOtrue(:,t,:);
            else
                dem_running(:,t+1,:) = BOtrue(:,t,:);
            end
      # No need to update demand becuase backorder shortage already spoken for. It is enroute, no need to see it in the demand matrix. 
        
      
      
      
      # Average amount of lag per part
      # BOtrue accounts for 1 time period for each period
      # BOlag can account for multiple time periods depending on how long
      # the lag time was
      # Avg time / part = [sum(BOtrue) + sum( sumproduct(BOlag, lag periods))] / part
      
      
      #  # NOT COMPLTE## Capture how long BOs exist for in terms of time ( I THINK I SHOULD ADD IN true BO and Exp Lags)
        qtyLag = (LagS>0).*x.CtS(:,:,1); # identofify QTY that is delayed(Lag BO)
        tLag = (qtyLag>0).*LagS; # the number of time periods that each qty was delayed
        
        #################
        # THis needs to be adjusted so that it is in terms of each product
        # so it can be scaled later on.
        # Standard Lag, plus true backorders, sup/sup transferred products,plus true expedited products
        time(t).totqtylag = sum(sum(sum(x.CtS.*LagS))) + sum(sum(x.CBO(:,2,:)))+ sum(sum(sum(((LagS+LagTransferExp).* x.CtransferExpFLOW))))+ sum(sum(LagExp'.*x.CtExp));
        # ALSO DO THIS FOR ITEMIZED PRODUCTS FOR SCALING
        time(t).totqtylagbyProduct = sum(sum(x.CtS.*LagS)) + sum(x.CBO(:,2,:)) + sum(sum(((LagS+LagTransferExp).* x.CtransferExpFLOW))) + sum(LagExp'.*x.CtExp);

      ## Save / Process outputs for current time period        
        # Save BO. 
        # NOTE***: Only the last time period BO is valid if you want to do
        # calculations later becauase the matrix builds and is complete at T
        
        # This saving of the backorders below is kind of redundent. I could
        # just save it at the end of all time periods. THis isnt used in
        # the future. It is just to visualize as the BO builds. r.BO
        # captures the endstate.
        time(t).BOtotal = BOtotal;
        time(t).BOlag = BOlag;
        time(t).BOtrue = BOtrue;
     ## Consolidate Costs for current time period                   
    time(t).TC = time(t).TMprodCost + time(t).TMtransCost + time(t).SupTransCost + time(t).InvCarCost+...
         time(t).AMProdCost+ time(t).ExpProdCost + time(t).ExpTransCost + time(t).ExpTransferCost;
  ##
   ## ITemized Costs for use in scaling
time(t).SupTransCostbyProduct = sum(sum(x.CtS.*CtS)) + sum(sum(x.CtransferExpFLOW.*CtransferExpFLOW)); # The expedited flow is the product that was transferred between supplies and was delivered to customers. There is no change in how it is deliverd in terms of time and cost.
time(t).AMProdCostbyProduct = sum(x.CpAM.*CpAM);
time(t).InvCarCostbyProduct = sum(sum(x.Ci.*Ci));

time(t).ExpProdCostbyProduct = x.CpExp.*CpExp ;# TM - Cust prod
time(t).ExpTransCostbyProduct = sum(x.CtExp.*CtExp); # TM - Cust actual trans

time(t).ExpTransferCostbyProduct = sum(sum(x.CtransferExp.*CtransferExp)); # Expedited lateral transfer

time(t).TMprodCostbyProduct = sum(reorder_qty.*CpTM); # Amount re-orderd, per supplier * the associated production cost
time(t).TMtransCostbyProduct =  sum(sum(TMtransCost,2)) ;  




 time(t).TCbyProduct = time(t).TMtransCostbyProduct + time(t).TMprodCostbyProduct + time(t).SupTransCostbyProduct + time(t).InvCarCostbyProduct +...
         time(t).AMProdCostbyProduct + time(t).ExpProdCostbyProduct + time(t).ExpTransCostbyProduct + time(t).ExpTransferCostbyProduct;
 
end # End time period loop

## Process Current demand realization outputs (consolidates time periods and capture for a single realization)
r(dr).time = time # Individiual timer period data stored by time periods for replication r
r(dr).TC = sum([time.TC]) + sum(AM_force*AM_fixed_cost); # Totals the cost for all time periods of a single replication. Also, adds in a one time cost associated with AM locations.
r(dr).BO = sum(sum(BOtotal)); # Total of all types of backorders
r(dr).BOlag = sum(sum(BOlag)) ; 
r(dr).BOtrue = sum(sum(BOtrue));

r(dr).ExpF = sum([time.ExpF]);
r(dr).ExpProd = sum([time.ExpCp]);
 
r(dr).totqtylag = sum([time.totqtylag]);
r(dr).y = y # Inventory Carried
r(dr).AM_loc_used = AM_loc_used;
r(dr).AM_qty_used = AM_qty_used;

r(dr).totqtylag = sum([time.totqtylag]); # IS THIS CORRECT
#*** NEED to think about what  is captured above.
#  Amount of AM produced, types o2 f backorders, etc.


# Scaled stuff
r(dr).TCbyProduct = sum([time.TCbyProduct]);
r(dr).AMfixedcost = sum(AM_force*AM_fixed_cost) ;# will be added to TCbyProduct later
r(dr).totqtylagbyProduct = sum([time.totqtylagbyProduct]); # IS THIS CORRECT

r(dr).rdel = sum(sum([time.del])); # by product total deliverd per realization

end # End demand realization loop
## Process Network design outputs (consolidates demand replications)
NetworkDes(nd).NetworkDesign = AM_force;
NetworkDes(nd).meanTC = mean([r.TC]);
NetworkDes(nd).meanBO = mean([r.BO]) ;# correct. Matches same as in plotting methods
NetworkDes(nd).meanBOtrue = mean([r.BOtrue]);


NetworkDes(nd).rTC = [r.TC]; # The total cost for each demand realization in a single network design
NetworkDes(nd).rBO = [r.BO] ;# The total # of BOs for every realization under the design.
NetworkDes(nd).rBOlag = [r.BOlag];
NetworkDes(nd).rBOtrue= [r.BOtrue] ;# The total # of BOs for every realization under the design.

NetworkDes(nd).rExpProd = [r.ExpProd]; 
NetworkDes(nd).rExpF = [r.ExpF]; # The total # of exp direct for every realization under the design.
NetworkDes(nd).r = r;
NetworkDes(nd).totqtylag = [r.totqtylag];
 
 #scaled stuff
NetworkDes(nd).TCbyProduct = [r.TCbyProduct]; # AM FIXED COST NOT INCLUDED YET
NetworkDes(nd).AMfixedcost = [r.AMfixedcost];
NetworkDes(nd).totqtylagbyProduct = [r.totqtylagbyProduct];
NetworkDes(nd).del = [r.rdel];

NetworkDes(nd).rAM_qty_used = [r.AM_qty_used]
NetworkDes(nd).ry = [r.y] # Inventory Carried

# Must save NetworkDes structure differently due to size
end # End network design loop (Do plots of all network designs now)
## Remove r to make structure smallers
fields = {'r'}
NetworkDes = rmfield(NetworkDes,fields)
## END EVALUATION: EVERYTHING THAT FOLLOWS IS PLOTTING / ANALYSIS

LoadError: UndefVarError: buildMILP not defined